 #  A Whale off the Port(folio)

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:
1. whale_returns.csv
2. algo_returns.csv
3. sp500_history.csv

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [2]:
whale_returns_csv = Path("../Resources/whale_returns.csv")
whale_returns = pd.read_csv(whale_returns_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)

algo_returns_csv = Path("../Resources/algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_csv, index_col= "Date", infer_datetime_format=True, parse_dates=True)

sp500_history_csv = Path("../Resources/sp500_history.csv")
sp500_history = pd.read_csv(sp500_history_csv, index_col="Date", infer_datetime_format=True, parse_dates=True)

sp500_history.head()

FileNotFoundError: [Errno 2] File ../Resources/whale_returns.csv does not exist: '../Resources/whale_returns.csv'

In [ ]:
whale_returns_clean = whale_return.dropna()

In [ ]:
whale_returns_clean.drop_duplicates()
what_returns_clean.head()

In [ ]:
# Count nulls
whale_returns.isnull().sum()

In [ ]:
# Drop nulls
whale_returns_clean = whale_returns.dropna()
whale_returns_cleans.isnull().sum()

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("algo_returns.csv")
algo_daily_returns = algo_returns.pct_change()

In [ ]:
# Drop nulls
algo_returns_clean = algo_returns.dropna()

In [ ]:
# Count nulls
algo_returns_clean.isnull().sum()

## S&P 500 Returns

Read the S&P500 Historic Closing Prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("sp500_history.csv")
sp500_history.head()

In [ ]:
# Check Data Types
sp500_history.dtypes

In [ ]:
# Fix Data Types
sp500_history['Close'] = sp500_history['Close'].str.replace('$', '')

In [ ]:
# Calculate Daily Returns
daily_returns_sp500 = sp500_history.astype(float).pct_change()
daily_returns_sp500.sort_index()
daily_returns_sp500.head()

In [ ]:
# Drop nulls
daily_returns_sp500.dropna()
daily_returns_sp500.head()

In [ ]:
# Rename Column
daily_returns_sp500 = daily_returns_sp500.rename(columns={
    "Close": "S&P 500",})
daily_returns_sp500.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat([whale_returns_clean, algo_returns_clean, daily_returns_sp500], axis ="columns", join = "inner")
combined_df.head()

---

# Portfolio Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance

Calculate and Plot the daily returns and cumulative returns. Does any portfolio outperform the S&P 500? 

In [ ]:
# Plot daily returns
combined_df.plot()

In [ ]:
# Plot cumulative returns
cumulative_returns = (1 + cumulative_df).cumprod()
cumulative_returns.plot()
print(cumulative_returns.tails())

## Risk

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

In [ ]:
# Box plot to visually show risk
combined_df.boxplot(figsize = (20,10))

In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. 
# Which portfolios are riskier than the S&P 500?
combined_df.std()

In [ ]:
# Determine which portfolios are riskier than the S&P 500
By visual observation we can see that there are two portfolios that are riskier than the S&P 500.  Tiger Global Management(0.0109) and Berkshire Hathaway(0.0130)have higher standard deviations than the S&P(0.009)

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_varaince = (combined_df.var()*252)
annualized_std = np.sqrt(annualized_varaince)
annualized_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Plot the rolling standard deviation of the various portfolios along with the rolling standard deviation of the S&P 500 (consider a 21 day window). Does the risk increase for each of the portfolios at the same time risk increases in the S&P?
2. Construct a correlation table for the algorithmic, whale, and S&P 500 returns. Which returns most closely mimic the S&P?
3. Choose one portfolio and plot a rolling beta between that portfolio's returns and S&P 500 returns. Does the portfolio seem sensitive to movements in the S&P 500?
4. An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the ewm with a 21 day half-life.

In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
combined_df.rolling(windows=21).std().plot(figsize=(15,8))

In [ ]:
# Construct a correlation table
correlation = combined_df.corr()
correlation

In [ ]:
#heatmap
sns.heatmap(correlation, vmin=-1, vmax=1)

In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)
# (Your graph may differ, dependent upon which portfolio you are comparing)
covariance = combined_df['BERKSHIRE HATHWAY INC'].cov(combined_df['S&P500'])
covariance

In [ ]:
variance = combined_df['BERKSHIRE HATHAWAY INC'].var()
variance

In [ ]:
BERK_beta = covariance / variance
BERK_beta

---

## Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. (After all, if you could invest in one of two portfolios, each offered the same 10% return, yet one offered lower risk, you'd take that one, right?)

1. Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot.
2. Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

In [ ]:
# Calculate annualized Sharpe Ratios
sharpe_ratios = (combined_df.mean() * 252) / (combined_df.std() * np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind="bar", title="Sharpe Ratios")

On the basis of this performance metric, do our algo strategies outperform both 'the market' and the whales? Type your answer here:

---

# Portfolio Returns

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Visit [Google Sheets](https://docs.google.com/spreadsheets/) and use the in-built Google Finance function to choose 3-5 stocks for your own portfolio.
2. Download the data as CSV files and calculate the portfolio returns.
3. Calculate the returns for each stock.
4. Using those returns, calculate the weighted returns for your entire portfolio assuming an equal number of shares for each stock.
5. Add your portfolio returns to the DataFrame with the other portfolios and rerun the analysis. How does your portfolio fair?


## Your analysis should include the following:

- Using all portfolios:
 - The annualized standard deviation (252 trading days) for all portfolios.
 - The plotted rolling standard deviation using a 21 trading day window for all portfolios.
 - The calculated annualized Sharpe Ratios and the accompanying bar plot visualization.
 - A correlation table.
- Using your custom portfolio and one other of your choosing:
 - The plotted beta. 

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Google stock
goog_csv = Path("../Resources/GOOGL_data.csv")
goog_df = pd.read_csv(goog_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
goog_df.rename( columns={"Symbol": "GOOG1","NOCP":"GOOG"},inplace = True )
goog_df.drop(columns=['GOOG1'], axis=1, inplace = True)

In [ ]:
# APPLE stock
aapl_csv = Path("../Resources/AAPL_data.csv")
aapl_df = pd.read_csv(aapl_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
aapl_df.rename( columns={"Symbol": "AAPL1","NOCP":"AAPL"},inplace = True )
aapl_df.drop(columns=['AAPL1'], axis=1, inplace= True)

In [ ]:
# TESLA stock
tsla_csv = Path("../Resources/TSLA_data.csv")
tsla_df = pd.read_csv(cost_csv, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
tsla_df.rename( columns={"Symbol": "TSLA1","NOCP":"TSLA"},inplace = True )
tsla_df.drop(columns=['TSLA1'], axis=1,inplace = True)

In [ ]:
# Concatenate all stocks into a single DataFrame
combined_pt2 = pd.concat([goog_df,aapl_df,tsla_df], axis='columns', join='inner')
combined_pt2.head()

In [ ]:
# Reset the index
combined_pt2.sort_index(inplace=True)
combined_pt2.head()

In [ ]:
# Pivot so that each column of prices represents a unique symbol
# YOUR CODE HERE

In [ ]:
# Drop Nulls
combined_pt2.dropna()
combined_pt2.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Calculate weighted portfolio returns
weights = [1/3, 1/3, 1/3]
portfolio_returns = daily_returns.dot(weights)

custom = pd.DataFrame(portfolio_returns)
custom.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Add your "Custom" portfolio to the larger dataframe of fund returns
combined_pt3 = pd.concat([portfolio_returns, combined_df], axis='columns', join='inner')

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
combined_pt3.rename(columns= {0: "Custom"},inplace = True )
combined_pt3.dropna(inplace=True)
combined_pt3.sort_index()
combined_pt3.head()

## Re-run the performance and risk analysis with your portfolio to see how it compares to the others

In [ ]:
# Risk
combined_pt3.std()
annualized_variance_pt2 = (combined_pt3.var()*252)
annualized_std_pt2 = np.sqrt(annualized_variance_pt2)
annualized_std_pt2

In [ ]:
# Rolling
rolling_variance = combined_pt3['Custom'].rolling(window=21).var()
rolling_covariance = combined_pt3['Custom'].rolling(window=21).cov(combined_pt3['S&P 500'])
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.plot(figsize=(20, 10), title='Custom Beta')

In [ ]:
# Annualized Sharpe Ratios
sharpe_ratios_all = (combined_pt3.mean() * 252) / ((combined_pt3).std() * np.sqrt(252))
sharpe_ratios_all

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios_all.plot(kind="bar", title="Sharpe Ratios")

In [ ]:
# Create a correlation analysis
correlation = combined_pt3.corr()
correlation

In [ ]:
#HeatMap
sns.heatmap(correlation, vmin=-1, vmax=1)

In [ ]:
# Beta
# YOUR CODE HERE